In [44]:
import pandas as pd

#文件读取
file = 'jobs_4k.xls'
content = pd.read_excel(file)

#只取postion和skill，其它的不需要 
position_names = content['positionName'].tolist()
skill_lables = content['skillLables'].tolist()

#构建graph数据
from collections import defaultdict
skill_position_graph = defaultdict(list)
for p, s in zip(position_names, skill_lables):
	skill_position_graph[p] += eval(s)
#print(skill_position_graph)

#画图
import networkx as nx
import matplotlib.pyplot as plt
G = nx.Graph(skill_position_graph)
#设置中文字体为黑体
plt.rcParams['font.sans-serif'] = ['SimHei']
#用来正常显示负号
plt.rcParams['axes.unicode_minus'] = False

#随机选20个点
import random
sample_nodes = random.sample(postion_names, k=20)

sample_nodes_connections = sample_nodes

#对20个点，取skill
for p, skills in skill_position_graph.items():
	if p in sample_nodes:
		sample_nodes_connections +=skills

#print(sample_nodes_connections)

#画图
sample_graph = G.subgraph(sample_nodes_connections)
plt.figure(figsize=(50,30))
pos = nx.spring_layout(sample_graph, k=1)
#nx.draw(sample_graph, pos, with_labels=True,node_size=30, font_size=10)
# plt.show()

#用pageRank做分析

pr = nx.pagerank(G, alpha=0.9)
ranked_position_and_ability = sorted([(name, value) for name, value in pr.items()], key = lambda x:x[1], reverse = True )
#print(ranked_position_and_ability)

#取所有的内容
X_content = content.drop(['salary'], axis = 1)
target = content['salary'].tolist()


X_content['merged'] = X_content.apply(lambda x: ''.join(str(x)), axis= 1)
print(X_content['merged'][0])
X_string = X_content['merged'].tolist()

import jieba
import re

#进行分词
def get_one_row_job_string(x_string_row):
	job_string = ''
	for i, element in enumerate(x_string_row.split('\n')):
		if len(element.split()) == 2:
			_, value = element.split()
			if i==0:
				continue
			job_string += value
	return job_string

#对target进行取值。
def token(string):
	return re.findall('\w+', string)


cutted_X = []
for i, row in enumerate(X_string):
	job_string = get_one_row_job_string(row)
	cutted_X.append(' '.join( list(jieba.cut(''.join(token(job_string))))    ) )
#print(cutted_X[0])

#tfidf进行训练
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(cutted_X)
import numpy as np

target_numical = [np.mean( list(map(float,re.findall('\d+',s) ) )) for s in target   ]
#print(target_numical)
Y = target_numical

from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=2)
model.fit(X,Y)

#进行预测
def predict_by_label(test_string, model):
	test_words = list(jieba.cut(test_string))
	test_vec = vectorizer.transform(test_words)
	y_pred = model.predict(test_vec)
	return y_pred[0]

test = '测试 北京 3年 专科'
print(test, predict_by_label(test, model))


test2 = '测试 北京 4年 专科'

print(test2, predict_by_label(test2, model))
test3 = '算法 北京 4年 本科'

print(test3, predict_by_label(test3, model))
test4 = 'UI 北京 4年 本科'

print(test4, predict_by_label(test4, model))
persons = [
"广州Java本科3年掌握大数据",
"沈阳Java硕士3年掌握大数据", 
"沈阳Java本科3年掌握大数据", 
"北京算法硕士3年掌握图像识别",
]
for p in persons:
	print("{} for {}".format(p, predict_by_label(p, model)))


id                                                                   3
positionName                                                    财务产品经理
district                                                           海淀区
stationname                                                         上地
jobNature                                                           全职
companyLabelList                      ['节日礼物', '年底双薪', '定期体检', '七险一金']
industryField                                               物流丨运输,软件开发
companySize                                                  500-2000人
skillLables                                      ['产品设计', '后台', 'B2B']
createTime                                         2020-05-15 15:10:02
companyFullName                                         北京福佑多多信息技术有限公司
workYear                                                          1-3年
education                                                           本科
positionAdvantage                                     福利待遇好、创业黑马、技术氛围好
url   

<Figure size 3600x2160 with 0 Axes>